In [1]:
import requests
import csv

In [2]:
# parse arguments
input_file = "../examples/run_input.csv"
output_file = "../examples/run_output_v2.csv"

# read smiles list
smiles_list = []
with open(input_file, "r") as f:
    reader = csv.reader(f)
    next(reader)
    for r in reader:
        smiles_list += [r[0]]

In [3]:
smiles_list

['COC(=O)[C@@H](N)CS',
 'CCCCNc1ccc(cc1)C(=O)OCCOCCOCCOCCOCCOCCOCCOCCOCCOC',
 'CCOCCOC(=O)\\C=C\\c1ccc(OC)cc1']

In [7]:
import requests

baseUrl = 'https://admetlab3.scbdd.com'
api = '/api/admetCSV'
url = baseUrl + api
with open("tmp.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["SMILES"])
    for s in smiles_list:
        writer.writerow([s])

# Open the file in binary mode and upload it
with open("tmp.csv", "rb") as f:
    files = {'file': ("tmp.csv", f, "text/csv")}
    response = requests.post(url, files=files)

print("Status:", response.status_code)
print("Response:", response.text)

Status: 400
Response: {"detail": "Cannot parse request body (Expecting value: line 1 column 1 (char 0))"}


In [ ]:

import requests

url = 'https://admetlab3.scbdd.com/api/single/admet'
param = {'SMILES': "CC(C)OC(=O)CC(=O)CSc1nc2c(cc1C#N)CCC2"}

response = requests.post(url, json=param)

print("Status:", response.status_code)
print("Raw Response:", response.text)

if response.status_code == 200:
    try:
        data = response.json()
        print("Prediction:", data.get('data', data))
    except Exception as e:
        print("JSON parsing error:", e)

Status: 500
Raw Response: Traceback (most recent call last):
  File "/root/miniconda3/envs/admetlab3/lib/python3.9/site-packages/ninja/operation.py", line 107, in run
    result = self.view_func(request, **values)
  File "/home/websites/admetlab3/admetlab3-backend/service/api.py", line 293, in calSingleSMILES
    tmp_dataframe = (result[columns])
  File "/root/miniconda3/envs/admetlab3/lib/python3.9/site-packages/pandas/core/frame.py", line 3899, in __getitem__
    indexer = self.columns._get_indexer_strict(key, "columns")[1]
  File "/root/miniconda3/envs/admetlab3/lib/python3.9/site-packages/pandas/core/indexes/base.py", line 6115, in _get_indexer_strict
    self._raise_if_missing(keyarr, indexer, axis_name)
  File "/root/miniconda3/envs/admetlab3/lib/python3.9/site-packages/pandas/core/indexes/base.py", line 6179, in _raise_if_missing
    raise KeyError(f"{not_found} not in index")
KeyError: "['BSEP'] not in index"



In [82]:
import requests

baseUrl = 'https://admetlab3.scbdd.com'

if __name__ == '__main__':
    api = '/api/single/admet'
    url = baseUrl + api
    param = {'SMILES': "CCC"}

    response = requests.post(url, json=param)
    print(response)
    if response.status_code == 200:
        data = response.json()['data']
        print(data)


<Response [500]>


In [25]:
import json

import requests
import pandas as pd

baseUrl = 'https://admetlab3.scbdd.com'


def transform(data):
    resultList = []
    for mol in data['data']:
        if not mol['data']:
            # Invalid SMILES
            tmp = {'smiles': mol['smiles']}
        else:
            tmp = dict({'smiles': mol['smiles']})
            for _, admet in mol['data'].items():
                for endpoint in admet:
                    # endpoint is a dict
                    tmp[endpoint['name']] = endpoint['value']
        resultList.append(tmp)
    return pd.DataFrame(resultList).fillna('Invalid SMILES')


def divide_list(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]



api = '/api/admetCSV'
url = baseUrl + api
param = {
    'SMILES': []
}
n = 1000
smiles_list = ['CN1C2CCC1CC(OC(=O)c1cccn1C)C2', 'O=C(O)Nc1scnc1C(=O)Nc1nccs1',
                'COc1ccc(C=C(F)C(=O)c2cc(OC)c(OC)c(OC)c2)cc1'] * 25

for _, sublist in enumerate(divide_list(smiles_list, n)):
    param['SMILES'] = sublist

    response = requests.post(url, json=param)
    print(response.status_code)

    if response.status_code == 200:  # If access is successful
        data = response.json()
        # transform to csv file
        result = transform(data)
        result.to_csv('result' + str(_) + '.csv', index=False)


200


TypeError: 'NoneType' object is not subscriptable

In [28]:
response.text

'null'